In [37]:
import idx2numpy
import torchvision.transforms as transforms
from PIL import Image
from matplotlib import cm
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
import pandas as pd

In [49]:
# Show labels
labels = idx2numpy.convert_from_file('../data/train-labels-idx1-ubyte')
torch.tensor(np.array(labels, copy=True)).unique()

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=torch.uint8)

In [50]:
def prepare_test_data():
    # read test data
    t10k = idx2numpy.convert_from_file('../data/t10k-images-idx3-ubyte')
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    # transform test datasets
    new_test = []
    for x in t10k:
        new_test.append(transform_test(Image.fromarray(np.uint8(x))))

    # convert to tensor of proper size
    test_tensor = torch.Tensor(len(new_test), 28, 28)
    torch.cat(new_test, out=test_tensor)
    new_test = test_tensor.view(-1, 1, 28, 28)
    
    # create dataset
    test_data = TensorDataset(new_test)

    # create data loader
    batch_size = 1
    test_loader = torch.utils.data.DataLoader(
        test_data,
        batch_size=batch_size
    )
    return test_loader

test_loader = prepare_test_data()

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layers
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        # linear layers
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        # dropout
        self.dropout = nn.Dropout(p=0.2)
        # max pooling
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        # convolutional layers with ReLU and pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # flattening the image
        x = x.view(-1, 7 * 7 * 16)
        # linear layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.fc4(x)
        return x
    
model = Net()
model.load_state_dict(torch.load('../trained_model.pt'))

<All keys matched successfully>

In [39]:
model.eval()

output = []

for batch_idx, (data,) in enumerate(test_loader):
    output.append([batch_idx, torch.argmax(model(data)).item()])

In [48]:
Submission = pd.DataFrame(output, columns=['Id','Class'])
Submission.to_csv('kaggle.out.csv', index=False)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])